In [2]:
import pandas as pd
from datetime import timedelta, datetime

Only one row has duplicated data in "address", but it's identifiable by "block hash"
Create constraint in postgres to prevent duplicates


In [3]:
path = "C:/Users/Adriano/Downloads/bquxjob_1a93124d_19118612d01.csv"

In [4]:
df_full = pd.read_csv(path)

In [20]:
print(df_full.info())
print(len(df_full))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46897 entries, 0 to 46896
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   address          46897 non-null  object        
 1   symbol           42838 non-null  object        
 2   name             42991 non-null  object        
 3   decimals         43455 non-null  float64       
 4   total_supply     45725 non-null  object        
 5   block_timestamp  46897 non-null  datetime64[ns]
 6   block_number     46897 non-null  int64         
 7   block_hash       46897 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 2.9+ MB
None
46897


In [6]:
max = df_full["block_timestamp"].max()
min = df_full["block_timestamp"].min()


print(max)
print(min)

date_format = "%Y-%m-%d" # %H:%M:%S.%f"

one_week = datetime.strptime(max[:10], date_format)+timedelta(days=-7)
print(one_week)

2024-08-03 06:46:23.000000 UTC
2016-01-28 14:01:48.000000 UTC
2024-07-27 00:00:00


In [7]:
df_dt_parsed = df_full
df_dt_parsed['block_timestamp'] = df_dt_parsed['block_timestamp'].map(lambda x: datetime.strptime(x[:10], date_format))

In [8]:
df_dt_parsed['block_timestamp'].describe()

count                            46897
mean     2019-04-16 13:52:36.764825088
min                2016-01-28 00:00:00
25%                2018-04-04 00:00:00
50%                2018-11-04 00:00:00
75%                2019-12-10 00:00:00
max                2024-08-03 00:00:00
Name: block_timestamp, dtype: object

In [9]:
df_filtered = df_dt_parsed[df_dt_parsed['block_timestamp'] >= one_week]

In [10]:
df_final = df_filtered.groupby("block_timestamp").size().reset_index(name="count")
print(df_final)

  block_timestamp  count
0      2024-07-27      8
1      2024-07-28     16
2      2024-07-29      9
3      2024-07-30      2
4      2024-07-31      3
5      2024-08-01      2
6      2024-08-02      2
7      2024-08-03      5


In [11]:
print(len(df_dt_parsed.drop_duplicates()))

46897


In [19]:
print(df_dt_parsed.groupby("address").size().reset_index(name="count").sort_values(by="count", ascending=False))

                                          address  count
46731  0xff0cca1818ddc0d66b53a96a9aa8bda9043e1437      2
0      0x000000030ef899748633e152920068c75a5f48b6      1
31266  0xaa7efc50c05f5440cca514ed175e286b175c486f      1
31258  0xaa6c4ef4856e3def10f2896facad4659f08b4cd2      1
31259  0xaa6da59f02ac6335ff0dbb896323181a8431a675      1
...                                           ...    ...
15633  0x543bf6d8f1afba861b0ef178fa1f5c4ea9ebac09      1
15634  0x5440ba85e9a2c380025e663a89662ab04dff3248      1
15635  0x54420565273f961d8b8c8f74b21aca7d85d44a59      1
15636  0x54437603c79fed354bc26b6ce051b487fab187fc      1
46895  0xffffe5b9cb42b4996997c9262f38d5f1f6a86fb7      1

[46896 rows x 2 columns]


In [13]:
print(df_dt_parsed[df_dt_parsed["address"] == "0xff0cca1818ddc0d66b53a96a9aa8bda9043e1437"])

                                          address symbol        name  \
17643  0xff0cca1818ddc0d66b53a96a9aa8bda9043e1437  yLink  Yearn Link   
39439  0xff0cca1818ddc0d66b53a96a9aa8bda9043e1437  yLink  Yearn Link   

       decimals                  total_supply block_timestamp  block_number  \
17643      18.0  1000000000000000000000000000      2021-03-29      12133335   
39439      18.0  1000000000000000000000000000      2021-03-29      12133416   

                                              block_hash  
17643  0xf6d43c38f4b7d2b65f6b4a8e5932dee72ea9cd0e71cf...  
39439  0x488bfc92cfd0166790f4c197d790648d5152d27d7470...  
